In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash
from dash import dcc, html
from pandas_datareader import wb
from datetime import datetime, timedelta
from dash_bootstrap_templates import load_figure_template
from dash.dependencies import Input, Output
from dash import dash_table


### Data

Index Data

In [2]:
# Get index tickers and names
tickers =  {
    '^N225'  : 'Nikkei 225',
    '^HSI' : 'Hang Seng',
    '^DJI'  : 'Dow Jones',
    '^NDX' : 'NASDAQ 100',   
}

In [3]:
end = int(datetime.today().timestamp())
start = int((datetime.today() - timedelta(360*3)).timestamp())

In [4]:
# List for storing temporary dfs
temp = []

# Loop over the tickers
for ticker in tickers.keys():

    # Define url
    url = 'https://query1.finance.yahoo.com/v7/finance/download/' + ticker + '?period1=' + str(start) + '&period2=' + str(end) + '&interval=1d&events=history&includeAdjustedClose=true'

    # Import data from url
    dfTemp = pd.read_csv(url)
    
    # Add ticker as column
    dfTemp['ticker'] = ticker
    
    # Append to list
    temp.append(dfTemp)

In [5]:
# Concat all dfs into single df
df = pd.concat(temp)

# Convert Date to datetime and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)

print(df['ticker'].nunique())
df.head()

4


,Open,High,Low,Close,Adj Close,Volume,ticker
Date,,,,,,,
2020-06-03,22649.009766,22818.869141,22462.679688,22613.759766,22613.759766,94100000,^N225
2020-06-04,22885.140625,22907.919922,22501.810547,22695.740234,22695.740234,92000000,^N225
2020-06-05,22613.080078,22865.880859,22563.560547,22863.730469,22863.730469,85600000,^N225
2020-06-08,23121.980469,23178.099609,23028.619141,23178.099609,23178.099609,108500000,^N225
2020-06-09,23135.789063,23185.849609,22933.140625,23091.029297,23091.029297,91500000,^N225


In [6]:
# drop adjusted close
df.drop('Adj Close', axis = 1, inplace = True)
df.drop('Volume', axis = 1, inplace = True)

# round to 3 decimals
df = df.round(3)

### Initialising Callbacks

In [7]:
options = [
    {'label' : 'Closing', 'value' : 'Close'},
    {'label' : 'High', 'value' : 'High'},
    {'label' : 'Low', 'value' : 'Low'},
    {'label' : 'Open', 'value' : 'Open'},
]

In [8]:
# Assume you have a list of colors as long as your tickers list
colors = ['red', 'blue', 'green', 'orange', 'purple', 'yellow', 'black', 'pink']

# Create a color map dictionary
color_map = {ticker: color for ticker, color in zip(tickers.keys(), colors)}

In [9]:
def add_rangeselector(figure):
    figure.update_layout(
            xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                             label="1m",
                             step="month",
                             stepmode="backward"),
                        dict(count=6,
                             label="6m",
                             step="month",
                             stepmode="backward"),
                        dict(count=1,
                             label="1y",
                             step="year",
                             stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"
            )
        )

### Application

In [10]:
dbcCSS = 'https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates@V1.0.2/dbc.min.css'

In [11]:
app = JupyterDash(external_stylesheets = [dbc.themes.BOOTSTRAP, dbcCSS])

app.layout = dbc.Container(
    children = [
        
        # Header
        html.H1('BAN438 Big Boy Index Viewer',
                style={
                    'fontFamily': 'Arial',  
                    'fontWeight': 'bold',  
                    'fontStyle': 'italic',  
                    'fontSize': '50px',  
                    }
                ),

        html.P('Yahoo Finance Data visualised with Python Dash',
                style={
                    'fontFamily': 'Arial',
                    'fontSize': '20px',
                    }
                ),
        
        dbc.Row(
            children = [
                ## Radio Buttons
                dbc.Col(
                    dcc.RadioItems(
                        options = options, 
                        value = 'Close',     # component_property for input
                        id = 'my_input',     # component_id for input
                        labelStyle = {'margin-right': '20px'}, # adjust spacing between label and input
                        style = {'fontSize': '15px',
                               'textAlign':'left'} # adjust display                                             
                    ),
                    width = 6,   # half the width of the container
                ),

                ## Currently Viewing: X
                dbc.Col(
                    html.P(
                        children = '',    # component_property         
                        id = 'my_output', # component_id for output, 
                        style = {'fontSize': '15px',
                                 'fontFamily': 'Arial',
                                 'textAlign':'right'} # adjust font size
                    ),
                    width = 6,   # half the width of the container
                ),
            ]
        ),

        dbc.Row(
            children = [
                dbc.Col(
                    # Dropdown menu to select index
                    dcc.Dropdown(
                        options = [{'label' : tickers[ticker], 'value' : ticker} for ticker in tickers.keys()],
                        value = '^N225', # default value
                        id = 'index_dropdown',
                        style = {'fontSize': '15px',
                                'textAlign': 'left',
                                'width': '50%'}
                    ),
                    width = 12,   # half the width of the container
                ),
            ]
        ),

        html.Br(), # line break
        
        dbc.Row(
            children = [
                dbc.Col(
                    ### Line graph of chosen option of chosen index 3/4th of the container
                    dcc.Graph(
                        id = 'main_graph',
                        figure = px.line(df, x = df.index, y = 'Close', color = 'ticker')
                    ),
                    width = 8,
                ),
                
                dbc.Col(
                    # Logo of NHH
                    html.Img(
                        src = 'https://www.nhh.no/contentassets/dab910c2b4b441648c82105332b974ac/nhh_logo_1f_positiv_helblaa.png',
                        style = {'height': '360px', 'width': '180px', 'margin-left': '75px'}
                    ),
                ),

                dbc.Col(
                    dcc.Graph(
                        id = 'second_graph',
                        ### Graph of selected column from the radio buttons showing all tickers
                        figure = px.line(df, x = df.index, y = 'Close', color = 'ticker')
                    ),
                    width = 8,
                ),

                 ### Table of chosen index 1/4th of the container
                dbc.Col(
                    html.Div(
                        dbc.Table(id='table', striped = True, bordered = True, hover = True),
                        style={'maxHeight': '300px', 'overflowY': 'auto', 'overflowX': 'hidden', 'marginTop': '80px'}
                    ),
                    width = 4,
                ),
                
            ]
        ),
    ],
    
    className = 'dbc',

    style = { # all style elements are written in camelCase, while the properties are written in kebab-case
        'textAlign' : 'left',
        'backgroundColor' : '#BDB7AB',       # change background color using a color string
        'color' : '#363636',               # change text color using a Hex color code
    },
)

@app.callback(
    [Output(component_id='my_output', component_property='children'),  # link the output from the function with html.P
     Output(component_id='main_graph', component_property='figure'),  # link the output from the function with dcc.Graph
     Output(component_id='second_graph', component_property='figure'),  # link the output from the function with the second dcc.Graph
     Output(component_id='table', component_property='children')],  # link the output from the function with dbc.Table
    [Input(component_id='my_input', component_property='value'),  # link the input to the function with dcc.RadioItems
     Input(component_id='index_dropdown', component_property='value')]  # link the input to the function with dcc.Dropdown
)

def metricUpdate(choice, selected_ticker):
    # Update the text
    output_text = str(choice) + ' is currently being viewed.'

    # Update the dataframe based on the selected ticker
    df_filtered = df[df['ticker'] == selected_ticker]

    # Drop the 'ticker' column
    filt_table = df_filtered.drop(columns=['ticker'])

    # Update the graph
    fig = px.line(df_filtered, x=df_filtered.index, y=choice, color='ticker', title = tickers[selected_ticker])

    # Define color map
    color_map = {'^N225': 'red', '^HSI': 'blue', '^OMX': 'green', '^DJI': 'purple', 
                 '^N100': 'red', '^GSPC': 'blue', '^NDX': 'green', 'OSEBX.OL': 'purple'}

    fig.for_each_trace(lambda trace: trace.update(line=dict(color=color_map[trace.name])))

    # Change the background color
    fig.update_layout(
        paper_bgcolor='#BDB7AB',  # this changes the color outside the plot
        plot_bgcolor='#D6CFC7',  # this changes the color of the plot background
    )

    # Update the table
    table = dbc.Table.from_dataframe(filt_table, striped=True, bordered=True, hover=True)

    # Update the second graph
    fig2 = px.line(df, x=df.index, y=choice, color='ticker', title = 'All Indices ' + str(choice))

    # Use the same color map and layout for the second graph
    fig2.for_each_trace(lambda trace: trace.update(line=dict(color=color_map[trace.name])))

    fig2.update_layout(
        paper_bgcolor='#BDB7AB',  # this changes the color outside the plot
        plot_bgcolor='#D6CFC7',  # this changes the color of the plot background
        yaxis=dict(range=[800, 40000])  # set the Y-axis range here
    )

    add_rangeselector(fig)
    add_rangeselector(fig2)

    return output_text, fig, fig2, table

In [12]:
app.run_server()

Dash app running on http://127.0.0.1:8050/
